In [ ]:
# default_exp core

# Load Roster Table

> Module to load the roster table.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
import pandas as pd
from pathlib import Path
import re


In [ ]:
data_path = Path.cwd() / "data"
# roster_path = data_path / "202201_ZIB_Jan_anon.xlsx"
roster_path = data_path / "private" / "202201_ZIB_Jan.xlsx"


In [ ]:
day_pattern = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
date_pattern = re.compile(r"\d{2}")


def daycol(col):
    return day_pattern.search(col)


In [ ]:
# TODO: Should use daycols for variable length months
df = pd.read_excel(roster_path, skiprows=0, header=1, index_col=0)
roster = (
    df.dropna().rename_axis("Name").loc[:, [col for col in df.columns if daycol(col)]]
)
roster

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from typing import Any
from ics import Calendar, Event


@dataclass
class ShiftProperties:
    name: str
    starting_hour: timedelta
    duration: timedelta


@dataclass
class Shift:
    properties: ShiftProperties
    date: datetime

    def __post_init__(self):
        self.beginning: datetime = self.date + self.properties.starting_hour


sp = ShiftProperties(
    name="Früh0", starting_hour=timedelta(hours=7), duration=timedelta(hours=8)
)
s = Shift(properties=sp, date=datetime(2022, 1, 1, 7, 0))


In [ ]:
_abbr2shiftproperties: Any = {
    "(/)": None,
    "2M": None,
    "3g": None,
    "F0": ShiftProperties(
        name="Früh0", starting_hour=timedelta(hours=8), duration=timedelta(hours=10)
    ),
    "F1": ShiftProperties(
        name="Früh1", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "F2": ShiftProperties(
        name="Früh2", starting_hour=timedelta(hours=8), duration=timedelta(hours=12)
    ),
    "FB": ShiftProperties(
        name="Fortbildung",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
    "N1": ShiftProperties(
        name="Nacht1", starting_hour=timedelta(hours=22), duration=timedelta(hours=10)
    ),
    "N2": ShiftProperties(
        name="Nacht2", starting_hour=timedelta(hours=20), duration=timedelta(hours=12)
    ),
    "S1": ShiftProperties(
        name="Spät1", starting_hour=timedelta(hours=15), duration=timedelta(hours=8)
    ),
    "U": ShiftProperties(
        name="Urlaub", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "ZA": ShiftProperties(
        name="Zeitausgleich",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
}


In [ ]:
@dataclass
class Roster:
    shifts: list[Shift]
    name: str = "Jane Doe"
    _year: int = 2022
    _month: int = 1  # TODO: Read from Excel
    _dayp = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
    _datep = re.compile(r"\d{2}")

    @classmethod
    def from_dict(cls, input: dict[str, str]) -> "Roster":
        shifts = []
        for date_str, abbr in input.items():
            props = _abbr2shiftproperties[abbr]
            if props is None:
                continue
            date = datetime(
                year=cls._year,
                month=cls._month,
                day=int(cls._datep.search(date_str).group()),
            )
            shift = Shift(props, date=date)
            shifts.append(shift)
        return cls(shifts=shifts)

    def to_ics(self):
        c = Calendar()
        for shift in self.shifts:
            e = Event()
            e.name = shift.properties.name
            e.begin = shift.beginning
            e.duration = shift.properties.duration
            c.events.add(e)
        return c


In [ ]:
for name in roster.index:
    r = Roster.from_dict(roster.loc[name].to_dict())
    filename = f"{name.replace(',', '_').replace(' ', '')}.ics"
    with (data_path / "output" / filename).open('w') as f:
        f.write(str(r.to_ics()))
    